# RT-DETR: Real-Time Detection Transformer

## 소개

**RT-DETR**은 최초의 실시간 엔드-투-엔드 객체 탐지기입니다. 본 연구에서는 **RT-DETR-L**과 **RT-DETR-X** 두 가지 버전의 모델을 제안합니다.

## 성능

- **RT-DETR-L**은 COCO val2017에서 **53.0% AP**를 달성하며 T4 GPU에서 **114 FPS**의 속도를 보여줍니다.
- **RT-DETR-X**는 **54.8% AP**로, 동일한 규모의 YOLO 탐지기들을 속도와 정확도 모두에서 능가합니다.
- **RT-DETR-R50**은 **53.1% AP**와 **108 FPS**를 달성하여, DINO-Deformable-DETR-R50 모델을 정확도에서 **2.2% AP**로, 속도에서는 약 **21배** 빠르게 능가합니다.

## 비교

| 모델 | AP (COCO val2017) | FPS (T4 GPU) | 비고 |
| --- | --- | --- | --- |
| RT-DETR-L | 53.0% | 114 | - |
| RT-DETR-X | 54.8% | 74 | YOLO 탐지기 대비 우수함 |
| RT-DETR-R50 | 53.1% | 108 | DINO-Deformable-DETR-R50 대비 우수함 |


# requirements pip list
## 종속성 설치 파일들과 FHAILIB를 설치 합니다.

In [ ]:
import os
os.system("pip install fhailib")

## 데이터셋을 다운로드 합니다

In [ ]:
import os
from src.RT_DETR_util.RT_DETR_Tool import download_dataset

# 현재 작업 경로의 'dataset' 폴더 내에 데이터셋 다운로드 및 압축 해제
current_working_directory = os.getcwd()
dataset_path = os.path.join(current_working_directory, 'DATA/COCO')

download_dataset(dataset_path)

# I. 시작

`train_model`, `Convert_Onnx`, `inference`, `convert_labels_to_json` 함수들을 가져옵니다.


In [ ]:

#os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from src.RT_DETR_util.RT_DETR_Tool import train_model, Convert_Onnx, inferencer, convert_labels_to_json

current_working_directory = os.getcwd()
print(current_working_directory)

## II. 라벨 데이터 Json화

### txt가 들어있는 폴더내에 파일을 모두 검색해서 Json 파일에 로깅합니다.
파일 내에 있는 라벨형식은 YOLO 형식입니다 

In [16]:
from RT_DETR_util.RT_DETR_Tool import config_category2name
classes = config_category2name()

# 학습 폴더 Json 변환
txt_files_path = './DATA/COCO/labels'
images_path = './DATA/COCO/images'
output_json_path = './DATA/COCO/meta/train_annotations.json'

convert_labels_to_json(classes, txt_files_path, output_json_path, images_path)


## III. 설정 파일 및 전이학습 모델 설정
### 모델 학습을 실시합니다.
모델경로를 지정하면 전이학습을.

모델을 지정하지 않으면 초기부터 학습을 진행합니다. 

.\RT_DETR_util\configs\dataset\Dataset_Config.yml 내에 Class개수를 맞지 않게 설정하면 에러가 발생합니다.

In [ ]:
yml_path = "./src/RT_DETR_util/configs/rtdetr/rtdetr_r18vd_6x_coco.yml"
model_path = ""

if __name__ == "__main__":
    train_model(yml_path, model_path)

## VI. ONNX 변환
### PT 파일을 ONNX 모델로 변환합니다
yml 파일내의 class(nc)의 개수를 잘못 설정하면 에러가 발생합니다.

In [ ]:
model_path = "./DATA/COCO/result/2024-02-28_16-30-01/best.pt"
yml_path = "./src/RT_DETR_util/configs/rtdetr/rtdetr_r18vd_6x_coco.yml"
Convert_Onnx(yml_path, model_path)

## VII. 모델 추론
### onnx 모델을 이용하여 추론을 실시 합니다.
경로의 파일을 모두읽어 추론하는 모드와 이미지 하나만 추론하는 모드가 있습니다.

In [ ]:
image_path = 'DATA/COCO/images'
onnx_path  = "DATA/COCO/result/2024-02-28_16-30-01/onnx_model/best.onnx"

inferencer(image_path, onnx_path, thrh= 0.6, mode=True)
